In [ ]:
%load_ext autoreload
%autoreload 2

# TWS Real-Time Positions Export

This notebook connects to the IBKR Trader Workstation, fetches real-time positions
using the `IbkrTws` class, exports them as a pandas DataFrame, and validates
against the Pandera schema.

## Purpose

1. Validate that `IbkrTws.positions_df()` works with live TWS data
2. Inspect the resulting DataFrame columns and dtypes
3. Export the schema for refinement
4. Confirm the Pandera schema matches the live data shape

## Connect to TWS

In [ ]:
import pandas as pd
from ib_async import IB, util

util.startLoop()

host = "127.0.0.1"
port = 7496
clientId = 102

TWS_CONNECTION = IB().connect(host, port, clientId, timeout=30)

## Fetch Positions via IbkrTws

In [ ]:
from ngv_reports_ibkr.ibkr_tws import IbkrTws

tws = IbkrTws(ib=TWS_CONNECTION)
accounts = tws.get_accounts()
print(f"Managed accounts: {accounts}")

### Inspect raw Position objects

Before converting to a DataFrame, inspect the raw ib_async Position namedtuples
to understand the data shape.

In [ ]:
account_id = accounts[0]  # Update index if needed

raw_positions = tws.get_positions_for_account(account_id)
print(f"Found {len(raw_positions)} positions for {account_id}")

# Inspect first position object
if raw_positions:
    pos = raw_positions[0]
    print(f"\nPosition fields: account={pos.account}, position={pos.position}, avgCost={pos.avgCost}")
    print(f"Contract type: {type(pos.contract).__name__}")
    print(f"Contract: {pos.contract}")

### Export positions as DataFrame

Use `positions_df()` which expands contract fields and validates against the Pandera schema.

In [ ]:
positions_df = tws.positions_df(account_id)

if positions_df is not None:
    print(f"Positions DataFrame: {positions_df.shape[0]} rows x {positions_df.shape[1]} columns")
    positions_df
else:
    print("No positions found.")

## Inspect DataFrame Schema

Examine the dtypes and columns to confirm the schema matches expectations.
Use this output to refine the Pandera schema in `schemas/ibkr_tws_positions.py`.

In [ ]:
if positions_df is not None:
    print("Column dtypes:")
    print(positions_df.dtypes)
    print(f"\nTotal columns: {len(positions_df.columns)}")
    print(f"Columns: {positions_df.columns.tolist()}")

In [ ]:
if positions_df is not None:
    positions_df.T

### Export dtypes for schema refinement

Generate a markdown file documenting the DataFrame structure.
Use this to update the Pandera schema with any additional columns
discovered from live data.

In [ ]:
from ngv_reports_ibkr.dtype_exporter import export_dtypes_all

if positions_df is not None:
    export_dtypes_all(
        positions_df,
        "../data",
        prefix="ibkr_tws_positions",
        only_export=["md"],
    )

## Validate Against Pandera Schema

Run the schema validation explicitly to confirm the live data matches
the defined schema. If validation fails, the errors will show which
columns need schema updates.

In [ ]:
from ngv_reports_ibkr.schemas.ibkr_tws_positions import (
    ibkr_tws_positions_schema,
    validate_ibkr_tws_positions_lazy,
)

if positions_df is not None:
    try:
        validated_df = validate_ibkr_tws_positions_lazy(positions_df)
        print(f"Schema validation passed. {len(validated_df)} positions validated.")
    except Exception as e:
        print(f"Schema validation failed:\n{e}")
        print("\nUpdate schemas/ibkr_tws_positions.py with the correct dtypes.")

## Adapter Pattern

Demonstrate using the `TwsReportOutputAdapterPandas` adapter,
which follows the same pattern as `ReportOutputAdapterPandas`
for Flex reports.

In [ ]:
from ngv_reports_ibkr.ibkr_tws import TwsReportOutputAdapterPandas

adapter = TwsReportOutputAdapterPandas(tws=tws)

# Process all accounts
results = adapter.process_accounts()

for i, result in enumerate(results):
    account = accounts[i]
    pos_df = result["positions"]
    if pos_df is not None:
        print(f"{account}: {len(pos_df)} positions")
    else:
        print(f"{account}: no positions")

## Disconnect

In [ ]:
TWS_CONNECTION.disconnect()